In [1]:
import FinanceDataReader as fdr
import requests
import pandas as pd
import time
import ta
import openpyxl
import warnings
import cloudscraper
import tensorflow as tf
tf.config.experimental.set_visible_devices([], 'GPU')

from datetime import datetime, timedelta
from pykrx import stock
from pykrx import bond
from PublicDataReader import Fred
from marcap import marcap_data
from functools import reduce
from collections import OrderedDict
from defs_pred import Marcap, filter_df
from io import BytesIO
from openpyxl.utils.dataframe import dataframe_to_rows
from ics import Calendar, Event

In [2]:
# 캘린더 생성
cal = Calendar()

# 이벤트 생성
event = Event()
event.name = "신정"
event.begin = datetime(2024, 1, 1)  # 시작 날짜 설정 (년, 월, 일)
event.make_all_day()  # 하루 종일 이벤트로 설정

event1 = Event()
event1.name = '설날'
event1.begin = datetime(2024, 2, 9)  # 시작 날짜 설정 (년, 월, 일)
event1.make_all_day()  # 하루 종일 이벤트로 설정

event2 = Event()
event2.name = '설날(대체휴일)'
event2.begin = datetime(2024, 2, 12)  # 시작 날짜 설정 (년, 월, 일)
event2.make_all_day()  # 하루 종일 이벤트로 설정

event3 = Event()
event3.name = '3・1절'
event3.begin = datetime(2024, 3, 1)  # 시작 날짜 설정 (년, 월, 일)
event3.make_all_day()  # 하루 종일 이벤트로 설정

event4 = Event()
event4.name = '근로자의 날'
event4.begin = datetime(2024, 5, 1)  # 시작 날짜 설정 (년, 월, 일)
event4.make_all_day()  # 하루 종일 이벤트로 설정

event5 = Event()
event5.name = '어린이날(대체휴일)'
event5.begin = datetime(2024, 5, 6)  # 시작 날짜 설정 (년, 월, 일)
event5.make_all_day()  # 하루 종일 이벤트로 설정

event6 = Event()
event6.name = '석가탄신일'
event6.begin = datetime(2024, 5, 15)  # 시작 날짜 설정 (년, 월, 일)
event6.make_all_day()  # 하루 종일 이벤트로 설정

event7 = Event()
event7.name = '현충일'
event7.begin = datetime(2024, 6, 6)  # 시작 날짜 설정 (년, 월, 일)
event7.make_all_day()  # 하루 종일 이벤트로 설정

event8 = Event()
event8.name = '광복절'
event8.begin = datetime(2024, 8, 15)  # 시작 날짜 설정 (년, 월, 일)
event8.make_all_day()  # 하루 종일 이벤트로 설정

event9 = Event()
event9.name = '추석'
event9.begin = datetime(2024, 9, 16)  # 시작 날짜 설정 (년, 월, 일)
event9.make_all_day()  # 하루 종일 이벤트로 설정

event10 = Event()
event10.name = '추석'
event10.begin = datetime(2024, 9, 17)  # 시작 날짜 설정 (년, 월, 일)
event10.make_all_day()  # 하루 종일 이벤트로 설정

event11 = Event()
event11.name = '추석'
event11.begin = datetime(2024, 9, 18)  # 시작 날짜 설정 (년, 월, 일)
event11.make_all_day()  # 하루 종일 이벤트로 설정

event12 = Event()
event12.name = '개천절'
event12.begin = datetime(2024, 10, 3)  # 시작 날짜 설정 (년, 월, 일)
event12.make_all_day()  # 하루 종일 이벤트로 설정

event13 = Event()
event13.name = '한글날'
event13.begin = datetime(2024, 10, 9)  # 시작 날짜 설정 (년, 월, 일)
event13.make_all_day()  # 하루 종일 이벤트로 설정

event14 = Event()
event14.name = '성탄절'
event14.begin = datetime(2024, 12, 25)  # 시작 날짜 설정 (년, 월, 일)
event14.make_all_day()  # 하루 종일 이벤트로 설정

event15 = Event()
event15.name = '연말휴장일'
event15.begin = datetime(2024, 12, 31)  # 시작 날짜 설정 (년, 월, 일)
event15.make_all_day()  # 하루 종일 이벤트로 설정

# 캘린더에 이벤트 추가
cal.events.add(event)
cal.events.add(event1)
cal.events.add(event2)
cal.events.add(event3)
cal.events.add(event4)
cal.events.add(event5)
cal.events.add(event6)
cal.events.add(event7)
cal.events.add(event8)
cal.events.add(event9)
cal.events.add(event10)
cal.events.add(event11)
cal.events.add(event12)
cal.events.add(event13)
cal.events.add(event14)
cal.events.add(event15)

# 캘린더를 ics 파일로 저장
with open("2024휴장일.ics", "w") as f:
    f.write(str(cal))